# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
# Loading Environment variables
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Get PRICE_DATA
PRICE_DATA = os.getenv("PRICE_DATA")

# Find all .parquet files in PRICE_DATA directory
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
parquet_files

['../../05_src/data/prices/CDL/CDL_2019/part.0.parquet',
 '../../05_src/data/prices/CDL/CDL_2019/part.1.parquet',
 '../../05_src/data/prices/CDL/CDL_2017/part.0.parquet',
 '../../05_src/data/prices/CDL/CDL_2017/part.1.parquet',
 '../../05_src/data/prices/CDL/CDL_2016/part.0.parquet',
 '../../05_src/data/prices/CDL/CDL_2016/part.1.parquet',
 '../../05_src/data/prices/CDL/CDL_2020/part.0.parquet',
 '../../05_src/data/prices/CDL/CDL_2020/part.1.parquet',
 '../../05_src/data/prices/CDL/CDL_2018/part.0.parquet',
 '../../05_src/data/prices/CDL/CDL_2018/part.1.parquet',
 '../../05_src/data/prices/CDL/CDL_2015/part.0.parquet',
 '../../05_src/data/prices/CDL/CDL_2015/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2012/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2012/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2014/part.0.parquet',
 '../../05_src/data/p

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [11]:
# Write your code below.

import dask.dataframe as dd

dd_feat = dd.read_parquet(parquet_files).set_index("ticker")
# print(dd_feat.columns.tolist())

# Group by ticker and add lags for variables Close and Adj Close
dd_feat['Close_lag_1'] = dd_feat.groupby('ticker')['Close'].shift(1)
dd_feat['Adj_Close_lag_1'] = dd_feat.groupby('ticker')['Adj Close'].shift(1)

# Add returns
dd_feat['returns'] = (dd_feat['Close'] / dd_feat['Close_lag_1']) - 1

# Add hi_lo_range
dd_feat['hi_lo_range'] = dd_feat['High'] - dd_feat['Low']

# Reset index and persist
dd_feat = dd_feat.reset_index()
dd_feat = dd_feat.persist()

dd_feat.head()

/var/folders/xs/w60hxvws49nbndc6b_91qs4w0000gn/T/ipykernel_88807/746510254.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat['Close_lag_1'] = dd_feat.groupby('ticker')['Close'].shift(1)
/var/folders/xs/w60hxvws49nbndc6b_91qs4w0000gn/T/ipykernel_88807/746510254.py:10: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat['Adj_Close_lag_1'] = dd_feat.groupby('ticker')['Adj Close'].shift(1)


,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
0,A,2015-01-02,41.180000,41.310001,40.369999,40.560001,38.553932,1529200.0,A.csv,2015,NaN,NaN,NaN,0.940002
1,A,2015-01-05,40.320000,40.459999,39.700001,39.799999,37.831524,2041800.0,A.csv,2015,40.560001,38.553932,-0.018738,0.759998
2,A,2015-01-06,39.810001,40.020000,39.020000,39.180000,37.242188,2080600.0,A.csv,2015,39.799999,37.831524,-0.015578,1.000000
3,A,2015-01-07,39.520000,39.810001,39.290001,39.700001,37.736473,3359700.0,A.csv,2015,39.180000,37.242188,0.013272,0.520000
4,A,2015-01-08,40.240002,40.980000,40.180000,40.889999,38.867619,2116300.0,A.csv,2015,39.700001,37.736473,0.029975,0.799999


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [19]:
# Write your code below.

# Convert Dask dataframe to pandas dataframe
p_df_feat = dd_feat.compute()

# Add moving average of returns using a 10-day window
p_df_feat['returns_avg_10'] = p_df_feat.groupby('ticker')['returns'].rolling(10).mean().reset_index(0, drop=True)

p_df_feat.head(20)

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_avg_10
0,A,2015-01-02,41.180000,41.310001,40.369999,40.560001,38.553932,1529200.0,A.csv,2015,NaN,NaN,NaN,0.940002,NaN
1,A,2015-01-05,40.320000,40.459999,39.700001,39.799999,37.831524,2041800.0,A.csv,2015,40.560001,38.553932,-0.018738,0.759998,NaN
2,A,2015-01-06,39.810001,40.020000,39.020000,39.180000,37.242188,2080600.0,A.csv,2015,39.799999,37.831524,-0.015578,1.000000,NaN
3,A,2015-01-07,39.520000,39.810001,39.290001,39.700001,37.736473,3359700.0,A.csv,2015,39.180000,37.242188,0.013272,0.520000,NaN
4,A,2015-01-08,40.240002,40.980000,40.180000,40.889999,38.867619,2116300.0,A.csv,2015,39.700001,37.736473,0.029975,0.799999,NaN
5,A,2015-01-09,41.000000,41.000000,40.290001,40.590000,38.582447,1644900.0,A.csv,2015,40.889999,38.867619,-0.007337,0.709999,NaN
6,A,2015-01-12,40.610001,40.720001,39.950001,40.110001,38.126194,2770800.0,A.csv,2015,40.590000,38.582447,-0.011826,0.770000,NaN
7,A,2015-01-13,40.470001,40.700001,39.330002,39.549999,37.593884,2013100.0,A.csv,2015,40.110001,38.126194,-0.013962,1.369999,NaN
8,A,2015-01-14,39.029999,39.099998,38.209999,39.060001,37.128120,5134000.0,A.csv,2015,39.549999,37.593884,-0.012389,0.889999,NaN
9,A,2015-01-15,39.330002,39.410000,37.990002,38.009998,36.130066,2628900.0,A.csv,2015,39.060001,37.128120,-0.026882,1.419998,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.